In [1]:
!pip install backtesting ta-lib seaborn matplotlib

In [2]:
import numpy as np
import pandas as pd
import talib
from backtesting import Backtest, Strategy

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import mysql.connector
from mysql.connector import Error
conn = None
try:
    # Conéctate a la base de datos MySQL
    conn = mysql.connector.connect(
        host='localhost',
        database='backtesting',
        user='pruebas',
        password='contraseñaprueba'
    )

    if conn.is_connected():
        # Ejecuta la consulta para obtener los datos
        query = "SELECT * FROM btc_usdt"
        df_btc = pd.read_sql(query, conn)

        print("Datos obtenidos de la base de datos:")
        print(df_btc.head())

except Error as e:
    print(f"Error: {e}")
finally:
    if conn is not None and conn.is_connected():
        conn.close()
        print("Conexión a MySQL cerrada.")

#para cargar correctamente los datos a la estrategia las columnas deben estar con mayusculas
df_btc.columns = [col.capitalize() for col in df_btc.columns]
df_btc

Datos obtenidos de la base de datos:
            timestamp      open      high       low     close      volume
0 2024-04-25 06:00:00  64223.36  64391.99  64195.70  64288.00   363.60804
1 2024-04-25 07:00:00  64288.00  64379.85  63824.07  64017.99   951.17474
2 2024-04-25 08:00:00  64017.99  64176.86  63708.20  63948.78  1329.27338
3 2024-04-25 09:00:00  63948.78  64192.00  63809.13  63966.02   833.85634
4 2024-04-25 10:00:00  63966.03  63966.03  63337.25  63462.01  2418.87397
Conexión a MySQL cerrada.


/tmp/ipykernel_423833/3340958104.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_btc = pd.read_sql(query, conn)


,Timestamp,Open,High,Low,Close,Volume
0,2024-04-25 06:00:00,64223.36,64391.99,64195.70,64288.00,363.60804
1,2024-04-25 07:00:00,64288.00,64379.85,63824.07,64017.99,951.17474
2,2024-04-25 08:00:00,64017.99,64176.86,63708.20,63948.78,1329.27338
3,2024-04-25 09:00:00,63948.78,64192.00,63809.13,63966.02,833.85634
4,2024-04-25 10:00:00,63966.03,63966.03,63337.25,63462.01,2418.87397
...,...,...,...,...,...,...
995,2024-06-05 17:00:00,71312.78,71704.00,71231.48,71628.98,1367.90189
996,2024-06-05 18:00:00,71628.98,71748.22,70895.15,71096.81,1778.53896
997,2024-06-05 19:00:00,71096.80,71398.73,70871.20,71279.95,937.98536
998,2024-06-05 20:00:00,71279.95,71357.68,71160.00,71196.02,699.46084


In [4]:
class SMAPullbackStrategy(Strategy):
    n = 20

    def init(self):
        close = self.data.Close
        self.sma = self.I(talib.SMA, close, self.n)
    
    def next(self):
        price = self.data.Close[-1]
        sma = self.sma[-1]
        yesterdays_low = self.data.Low[-1]
        stop_loss = yesterdays_low
        take_profit = yesterdays_low + 1.5 *(price - stop_loss)

        if self.position.is_long:
            if price <= stop_loss or price >= take_profit:
                self.position.close()

        elif price > sma:
            print(stop_loss, ", ", take_profit)
            try:
                self.buy(sl=stop_loss, tp=take_profit)
            except:
                pass
        

        

In [ ]:
bt = Backtest(df_btc, SMAPullbackStrategy, cash=1000000, commission=0.002)
stats = bt.run()
#bt.plot()

In [7]:
print(stats)

Start                                     0.0
End                                     999.0
Duration                                999.0
Exposure Time [%]                        32.8
Equity Final [$]                 750210.54256
Equity Peak [$]                     1000000.0
Return [%]                         -24.978946
Buy & Hold Return [%]               10.497013
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -25.078156
Avg. Drawdown [%]                  -25.078156
Max. Drawdown Duration                  976.0
Avg. Drawdown Duration                  976.0
# Trades                                133.0
Win Rate [%]                        66.165414
Best Trade [%]                       1.157335
Worst Trade [%]                     -2.881003
Avg. Trade [%]                    

In [8]:
class SMAPullbackStrategy(Strategy):
    stop_loss = 0
    n1 = 20

    def init(self):
        price = self.data.Close
        self.sma = self.I(talib.SMA, price, self.n1)
    
    def next(self):
        price = self.data.Close[-1]
        low_price_prev_day = self.data.Low[-1]

        if price > self.sma[-1]:
            self.stop_loss = low_price_prev_day

            if price <= 2.4 * self.stop_loss:
                self.buy()
            

In [9]:
bt = Backtest(df_btc, SMAPullbackStrategy, cash=1000000, commission=0.002)

output = bt.optimize(maximize='Equity Final [$]', n1=range(5,40,1))

stats = bt.run()
bt.plot()

/tmp/ipykernel_423833/3251659232.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_btc, SMAPullbackStrategy, cash=1000000, commission=0.002)


GridPlot(id='p1466', ...)

[424031:424052:0612/070239.591657:ERROR:object_proxy.cc(576)] Failed to call method: org.freedesktop.DBus.ListActivatableNames: object_path= /org/freedesktop/DBus: org.freedesktop.DBus.Error.AccessDenied: An AppArmor policy prevents this sender from sending this message to this recipient; type="method_call", sender=":1.10089" (uid=1000 pid=424031 comm="/snap/brave/411/opt/brave.com/brave/brave /home/mj" label="snap.brave.brave (enforce)") interface="org.freedesktop.DBus" member="ListActivatableNames" error name="(unset)" requested_reply="0" destination="org.freedesktop.DBus" (bus)
[424031:424031:0612/070240.209023:ERROR:object_proxy.cc(576)] Failed to call method: org.xfce.ScreenSaver.GetActive: object_path= /org/xfce/ScreenSaver: org.freedesktop.DBus.Error.AccessDenied: An AppArmor policy prevents this sender from sending this message to this recipient; type="method_call", sender=":1.10093" (uid=1000 pid=424031 comm="/snap/brave/411/opt/brave.com/brave/brave /home/mj" label="snap.brav

In [10]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(GOOG, SmaCross, commission=.002,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single 

GridPlot(id='p2439', ...)

Abriendo en una sesión existente del navegador


[424031:424031:0612/073424.595866:ERROR:interface_endpoint_client.cc(722)] Message 1 rejected by interface blink.mojom.WidgetHost
[424031:424031:0612/085148.232148:ERROR:CONSOLE(0)] "Refused to load the font 'https://cdn.search.brave.com/serp/v2/_app/immutable/assets/inter-latin-400-normal.YtgfLPRn.woff2' because it violates the following Content Security Policy directive: "font-src 'self' data:".
", source: chrome-extension://mnojpmjdmbbfmejpflffifhffcmidifd/_generated_background_page.html (0)
[424031:424031:0612/085148.235266:ERROR:CONSOLE(0)] "Refused to load the font 'https://cdn.search.brave.com/serp/v2/_app/immutable/assets/inter-latin-400-normal.YtgfLPRn.woff2' because it violates the following Content Security Policy directive: "font-src 'self' data:".
", source: chrome-extension://mnojpmjdmbbfmejpflffifhffcmidifd/_generated_background_page.html (0)
[424031:424031:0612/085148.235398:ERROR:CONSOLE(0)] "Refused to load the font 'https://cdn.search.brave.com/serp/v2/_app/immutable